In [13]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table 
from dash.dependencies import Input, Output, State
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64
#CRUD Python module file name and class name
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#username and password and CRUD Python module name
username = "aacuser"
password = "notsecret"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwo')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#Dashboard layout
app.layout = html.Div([
    html.Hr(), #double horizontal rule for decoration/visual seperation
    html.Hr(),
    html.A([  #Grazioso Salvare Logo that links to snhu.edu when clicked
    html.Img( src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'10%', 'width':'10%', 'padding-left': '30px'})],href='www.snhu.edu'),
    html.Center(html.B(html.H5('Creator of DashBoard: Joann Carter'))),
    html.Hr(),  #double horizontal rule for decoration/visual seperation
    html.Hr(),
    html.Div([
     #interactive radio button filter for resetting the query, or searching for water resque, mountain resque, or disaster resque.
        html.P('Types of Rescue:'),
        dcc.RadioItems(
            id = 'filter-type',
            options =[
                {'label':'Reset Search Results', 'value':'reset'},
                {'label':'Water','value':'wtr'},
                {'label':'Mountain','value':'mtn'},
                {'label':'Disaster', 'value':'dstr'}
            ],
            value = 'reset', #radio buttons are initially set with no filter option in place
        ),
    ]),
    html.Hr(),
#data table layout
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        column_selectable=False,
        row_selectable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 3, #only show three rows at a time. Ensures the map and pie graph are in view of user
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that the pie chart and geolocation chart are side-by-side
      html.Div(className='row',
           style={'display' : 'flex'},
               children=[
          html.Div(
              id='graph-id',
              className='col s12 m6',

          ),
         html.Div(
             id='map-id',
             className='col s12 m6',
              )
          ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#filter interactive data table with radio button values
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
##Three types of animal queries:
#water resque = breed is 'Labrador Retriever Mix', 'Newfoundland','Chesapeake Bay Retriever', and sex_upon_outcome is intact Female and age_upon_outcome_in_weeks is 26-156 weeks
#mountain resque = German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, Rotweiler, Intact Male, 26 - 156 weeks
#disaster resque = Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, Rottweiler, Intact Male, 20-300 weeks

    if filter_type == 'wtr':
        df = pd.DataFrame.from_records(shelter.read({
                        "animal_type":"Dog",
                        "breed":{"$in":["Labrador Retriever Mix", "Newfoundland","Chesapeake Bay Retriever"]},
                        "sex_upon_outcome":"Intact Female",
                        "age_upon_outcome_in_weeks":{"$gte":26},
                        "age_upon_outcome_in_weeks":{"$lte":156}
                    }))
    elif filter_type == 'mtn':
        df = pd.DataFrame.from_records(shelter.read({
                        "animal_type":"Dog",
                        "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
                        "sex_upon_outcome":"Intact Male",
                        "age_upon_outcome_in_weeks":{"$gte":26},
                        "age_upon_outcome_in_weeks":{"$lte":156}
                    }))        
    elif filter_type == 'dstr':
        df = pd.DataFrame.from_records(shelter.read({
                        "animal_type":"Dog",
                        "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                        "sex_upon_outcome":"Intact Male",
                        "age_upon_outcome_in_weeks":{"$gte":20},
                        "age_upon_outcome_in_weeks":{"$lte":300}
                    }))        
    else: # if no filter is selected, all animals are rendered to the data table
        df = pd.DataFrame.from_records(shelter.read({}))        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    return (data,columns)
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
#this callback updates the pie chart based on the data rendered in the table
@app.callback(
    Output('graph-id', "children"),
   [Input('datatable-id', "derived_viewport_data")]
    
)
#pie chart
def update_graphs(values):
    dfChart = pd.DataFrame.from_dict(values)
    fig = px.pie(dfChart, names='breed')
    return [
        dcc.Graph(     
           figure = fig
        )    
    ]
#this callback updates the geolocation map based on the data rendered in the table
@app.callback(
        Output('map-id',"children"),
        [Input('datatable-id', "derived_viewport_data")], 
        ) 

def update_map(viewData):
#geolocation chart
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.iloc[0]
    return [
        dl.Map(center=[dff.loc['location_lat'], dff.loc['location_long']], zoom=10, style={'width': '500px', 'height': '500px'},children=[
                    dl.TileLayer(id="base-layer-id"),
#             # Marker with tool tip and popup
                    dl.Marker(position=[dff.loc['location_lat'], dff.loc['location_long']], 
                              children=[
                                    dl.Tooltip(dff.loc['breed']),
                                    dl.Popup([
                                         html.H1("Animal Name"),
                                         html.P(dff.loc['name'])
                                        ])

                                ])
            
                            
                        ])
              
        ]


app

here
